In [99]:
from tensorflow import keras
from tensorflow.keras import layers, models

import tensorflow as tf
import numpy as np
import imageio

import matplotlib.pyplot as plt

In [100]:
batch_size = 64
num_channels = 3
num_classes = 131
image_size = (100,100)
latent_dim = 128

directory = "fruit/fruits-360_dataset/fruits-360/Training/"

In [101]:
dataset = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42
)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

Found 67692 files belonging to 131 classes.


In [102]:
generator_input = layers.Input(shape=(latent_dim,))
discriminator_input = layers.Input(shape=(image_size[0], image_size[1], num_channels))
print(generator_input, "\n", discriminator_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name='input_54'), name='input_54', description="created by layer 'input_54'") 
 KerasTensor(type_spec=TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name='input_55'), name='input_55', description="created by layer 'input_55'")


In [103]:
generator = Sequential(name="Generator")
generator.add(layers.InputLayer(input_shape=(latent_dim,)))
generator.add(layers.Dense(25 * 25 * 256, use_bias=False))
generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU(alpha=0.2))
generator.add(layers.Reshape((25, 25, 256)))

generator.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU(alpha=0.2))

generator.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU(alpha=0.2))

generator.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))


In [104]:
discriminator = Sequential(name="Discriminator")

discriminator.add(layers.InputLayer(input_shape=(image_size[0], image_size[1], num_channels)))

discriminator.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.GlobalMaxPooling2D())

discriminator.add(layers.Dense(1, activation='sigmoid'))

In [105]:
gan_input = layers.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = models.Model(gan_input, gan_output)

In [106]:
generator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [108]:
# Training parameters
epochs = 10
batch_size = 64

In [110]:
# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Train Discriminator
    for _ in range(len(dataset) // batch_size):
        # Generate a batch of real images with labels
        real_images, real_labels = next(iter(dataset))

        # Generate a batch of fake images with labels
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        generated_labels = np.zeros((batch_size, 1))  # Fake labels

        # Train the discriminator on real images
        discriminator.trainable = True
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)

        # Train the discriminator on fake images
        d_loss_fake = discriminator.train_on_batch(generated_images, generated_labels)

        # Update the discriminator's weights
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # Real labels

    # Train the GAN (discriminator frozen)
    discriminator.trainable = False
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Save generated images at certain intervals if needed

# Optionally, save the generator model for future use
generator.save("cgan_generator.h5")


Epoch 1/10


2023-11-30 18:47:27.975587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [64,1]
	 [[{{node Placeholder/_1}}]]


Epoch 2/10


2023-11-30 18:47:30.794244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [64,1]
	 [[{{node Placeholder/_1}}]]


Epoch 3/10


...

2023-11-30 18:47:47.455365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [64,1]
	 [[{{node Placeholder/_1}}]]


Epoch 9/10


2023-11-30 18:47:50.238379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [64,1]
	 [[{{node Placeholder/_1}}]]


Epoch 10/10


2023-11-30 18:47:53.015450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [64,1]
	 [[{{node Placeholder/_1}}]]
